In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ENCODING_DICT = {
            'a': 0,
            'f': 1,
            'e': 2,
            'c': 3,
            'b': 4,
            'h': 5,
            'v': 6,
            'z': 7,
            '2': 8,
            'x': 9,
            'g': 10,
            'm': 11,
            'r': 12,
            'u': 13,
            'p': 14,
            's': 15,
            'd': 16,
            'n': 17,
            '6': 18,
            'k': 19,
            't': 20
            }

DECODING_DICT = {
            0: 'a',
            1: 'f',
            2: 'e',
            3: 'c',
            4: 'b',
            5: 'h',
            6: 'v',
            7: 'z',
            8: '2',
            9: 'x',
            10: 'g',
            11: 'm',
            12: 'r',
            13: 'u',
            14: 'p',
            15: 's',
            16: 'd',
            17: 'n',
            18: '6',
            19: 'k',
            20: 't'
            }

In [12]:
def imagepath_to_captcha(imagepath):
    return imagepath.split("_")[1].split(".")[0]

def to_onehot(captcha):
    label = np.zeros((len(captcha), len(ENCODING_DICT)))
    for index, char in enumerate(captcha):
        label[index, ENCODING_DICT[char]] = 1
    return tf.convert_to_tensor(label, tf.float32)
    # return tf.reshape(tf.convert_to_tensor(label, tf.float32), [-1])

def captcha_from_onehot(tensor):
    label = []
    for index in tf.argmax(tensor, 1).numpy():
        label.append(DECODING_DICT[index])
    return "".join(label)

# Instead of one hot, if we directly use the labels, the label becomes of size 6.
def to_labels(captcha):
    label = np.zeros(len(captcha))
    for index, char in enumerate(captcha):
        label[index] = ENCODING_DICT[char]
    return tf.convert_to_tensor(label, tf.float32)

def load_dataset(filepath):
    filenames = os.listdir(filepath)
    x, y = [], []
    for file in filenames:
        imagepath = filepath + "/" + file
        image = tf.io.read_file(imagepath)
        image = tf.image.decode_image(image)
        pixels = tf.image.convert_image_dtype(image, tf.float32)
        # To show images: 
        # plt.imshow(tf.reshape(pixels, image.shape))
        # plt.title('Image as Pixels')
        # plt.show()
        x.append(pixels)
        captcha = imagepath_to_captcha(file).lower()
        y.append(to_labels(captcha))
    return (tf.convert_to_tensor(x, tf.float32), tf.convert_to_tensor(y, tf.float32))


# Read input data

In [13]:
train_x, train_y = load_dataset("dataset/train")
test_x, test_y = load_dataset("dataset/test")

print("Training input shape:", train_x.shape)
print("Training output shape:", train_y.shape)
print("Testing input shape:", test_x.shape)
print("Testing output shape:", test_y.shape)

Training input shape: (8501, 50, 250, 3)
Training output shape: (8501, 6)
Testing input shape: (1500, 50, 250, 3)
Testing output shape: (1500, 6)


In [44]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(128, (2, 2), (2, 2), activation='relu', input_shape=(50, 250, 3)))
model.add(keras.layers.MaxPooling2D(2,2))

model.add(keras.layers.Conv2D(64, (2, 2), (2, 2), activation='relu'))
model.add(keras.layers.MaxPooling2D(2, 2))

model.add(keras.layers.Flatten())

# model.add(keras.layers.Dense(128, activation='relu'))
# model.add(keras.layers.Dense(64, activation='relu'))
# model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(6, activation='softmax'))


model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 25, 125, 128)      1664      
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 12, 62, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 6, 31, 64)         32832     
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 3, 15, 64)         0         
 ng2D)                                                           
                                                                 
 flatten_12 (Flatten)        (None, 2880)              0         
                                                                 
 dense_35 (Dense)            (None, 64)              

In [52]:
model.compile(
    optimizer = "adam",
    loss = 'categorical_crossentropy',
    # loss = 'mse',
    metrics=['accuracy']
)

# graphwiz doesn't install on gLinux ffs.
# tf.keras.utils.plot_model(model, show_shapes=True)


In [53]:
history = model.fit(train_x, train_y,
                    epochs = 5)

print(history.history)

Epoch 1/5
266/266 [==============================] - 28s 102ms/step - loss: 6709726.0000 - accuracy: 0.1869
Epoch 2/5
266/266 [==============================] - 29s 110ms/step - loss: 160284000.0000 - accuracy: 0.1820
Epoch 3/5
266/266 [==============================] - 29s 110ms/step - loss: 758322048.0000 - accuracy: 0.1822
Epoch 4/5
266/266 [==============================] - 27s 103ms/step - loss: 2078430592.0000 - accuracy: 0.1769
Epoch 5/5
266/266 [==============================] - 26s 99ms/step - loss: 4267939328.0000 - accuracy: 0.1797
{'loss': [6709726.0, 160284000.0, 758322048.0, 2078430592.0, 4267939328.0], 'accuracy': [0.1869191825389862, 0.18197858333587646, 0.18221385776996613, 0.17692036926746368, 0.17974355816841125]}


In [47]:
model.evaluate(test_x, test_y)

47/47 [==============================] - 2s 28ms/step - loss: 132.4942 - accuracy: 0.3500


[132.49417114257812, 0.3499999940395355]